In [1]:
# encoding: utf8
from __future__ import print_function
import multiprocessing
from joblib import Parallel, delayed

import argparse
import tqdm
import os
import glob
import pandas as pd

# trade
# area_code,transaction_amount,year_of_construction,transaction_year,legal_dong,apt_name,transaction_month,transaction_day,dedicated_area,jibun,floor,해제사유발생일,해제여부,si,gu,sigungu,area,dedicated_area_level,amount_per_area,transaction_date,description
# rent
# area_code,year_of_construction,transaction_year,legal_dong,deposit,apt_name,transaction_month,monthly_rent,transaction_day,dedicated_area,jibun,floor,si,gu,sigungu,sale_type,transaction_amount,area,dedicated_area_level,amount_per_area,transaction_date,description
COLS = {}
COLS['apt-trade'] = ['si', 'gu', 'sigungu', 'legal_dong', 'apt_name', 'transaction_amount', 'transaction_date', 'description',
                     'transaction_year', 'transaction_month', 'floor', 'dedicated_area', 'year_of_construction']
COLS['apt-rent'] = ['si', 'gu', 'sigungu', 'legal_dong', 'apt_name', 'transaction_amount', 'transaction_date',
                    'transaction_year', 'transaction_month', 'floor', 'dedicated_area', 'monthly_rent', 'deposit']


def preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    # preprocessing
    df['transaction_amount'] = df['transaction_amount'].astype(float)
    # df['transaction_amount'] = df['transaction_amount'].astype(
    # float).apply(lambda x: round(x / 10000, 2))
    # 2016-05-26 0:00:00
    df['transaction_date'] = pd.to_datetime(
        df['transaction_date'], format="%Y-%m-%d %H:%M:%S").dt.date
    df['transaction_year'] = df['transaction_year'].astype(object)
    df['transaction_month'] = df['transaction_month'].astype(object)
    df['year_of_construction'] = df['year_of_construction'].astype(int)
    df['floor'] = df['floor'].astype(int)
    df['dedicated_area'] = df['dedicated_area'].astype(float)

    if 'monthly_rent' in df.columns:
        df['monthly_rent'] = df['monthly_rent'].astype(int)
    return df
 

def tempFunc(df: pd.DataFrame) -> pd.DataFrame:
    df['mean_transaction_amount'] = df['transaction_amount'].mean()
    return df


def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)


def transaction_amount_year(df: pd.DataFrame) -> pd.DataFrame:
    df = df.set_index('apt_name')
    print(len(df))
    frames = []
    print(f' apt counts : {len(df.index)}')
    apt_names = df.index[:10]
    for apt_name in tqdm.tqdm(apt_names):
        data = df.loc[[apt_name]]
        # data = applyParallel(
        # data[['transaction_amount']].groupby(data.index), tempFunc)
        data = data[['transaction_year', 'transaction_amount']
                    ].groupby(data.index).mean()
        data = data.reset_index()
        frames.append(data)
    result_df = pd.concat(frames, axis=0)
    return result_df


# 현재 아파트별 면적/층별 거래가격

In [ ]:

def transaction_amount_year(df: pd.DataFrame) -> pd.DataFrame:
    df = df.set_index('apt_name')
    print(len(df))
    frames = []
    print(f'apt counts : {len(df.index)}')
    apt_names = df.index
    # apt_names = df.index
    for apt_name in tqdm.tqdm(apt_names):
        data = df.loc[[apt_name]].reset_index()
        # data = applyParallel(
        # data[['transaction_amount']].groupby(data.index), tempFunc)
        data = data[['apt_name','transaction_year', 'transaction_amount']
                    ].groupby(['apt_name','transaction_year']).mean()
        data = data.reset_index()
        data['transaction_amount'] = data['transaction_amount'].apply(lambda x: round(x / 10000,1))
        frames.append(data)
        # print(data)
    result_df = pd.concat(frames, axis=0)
    return result_df

In [62]:
def price_dedicatedarea_floor(df: pd.DataFrame) -> pd.DataFrame:
    # df = df.set_index('apt_name')
    frames = [] 
    apt_name_list = df['apt_name'].unique().tolist()
    
    for apt_name in tqdm.tqdm(apt_name_list):
        data = df[df['apt_name'] == apt_name] 
        data = data[['apt_name','dedicated_area','floor','transaction_amount','transaction_date']]
        data = data.sort_values('transaction_date', ascending=True).groupby(['floor','dedicated_area']).tail(1)     
        data = data.reset_index()
        frames.append(data)
    result_df = pd.concat(frames, axis=0)
    return result_df

In [59]:
df = pd.read_csv('../data_in/41135/201604.csv', usecols=COLS[trade_type])
df = preprocessing(df)

# apt_name_list = df['apt_name'].unique().tolist()
# data = df[df['apt_name'] == apt_name_list[0]]
price_dedicatedarea_floor(df)

100%|██████████| 144/144 [00:00<00:00, 305.04it/s]


,index,apt_name,dedicated_area,floor,transaction_amount,transaction_date
0,0,샛별마을(라이프),84.99,18,52500.0,2016-04-01
1,6,샛별마을(라이프),126.42,16,66500.0,2016-04-13
0,1,장안타운(건영),85.00,1,46000.0,2016-04-06
1,2,장안타운(건영),70.68,7,40500.0,2016-04-06
2,11,장안타운(건영),85.00,18,48200.0,2016-04-20
...,...,...,...,...,...,...
0,593,산운마을10단지(대광로제비앙),83.50,15,69000.0,2016-04-09
1,602,산운마을10단지(대광로제비앙),83.50,9,71000.0,2016-04-23
0,600,산운마을9단지(대방노블랜드)임대전환,59.93,7,57500.0,2016-04-18
0,601,산운마을5단지(한성필하우스),84.81,10,64000.0,2016-04-22


In [28]:
data[['apt_name','dedicated_area','floor','transaction_amount','transaction_date']]

,apt_name,dedicated_area,floor,transaction_amount,transaction_date
0,샛별마을(라이프),84.99,18,52500.0,2016-04-01
6,샛별마을(라이프),126.42,16,66500.0,2016-04-13


In [29]:
trade_type = 'apt-trade'
for area_code_dir in area_code_dirs: #tqdm.tqdm(area_code_dirs): 
    area_code = area_code_dir.split('/')[-1]
    print(area_code_dir)
    filelist = glob.glob(os.path.join(area_code_dir, '*.csv'))
    frames = []
    print(filelist[:1])
    print(filelist)
    for filepath in tqdm.tqdm(filelist):
        frame = pd.read_csv(filepath, usecols=COLS[trade_type])
        frames.append(frame)
    print(len(df))
    df = pd.concat(frames, axis=0)
    df = preprocessing(df)

8%|▊         | 15/182 [00:00<00:01, 147.45it/s]../data_in/41135
['../data_in/41135/201604.csv']
['../data_in/41135/201604.csv', '../data_in/41135/201610.csv', '../data_in/41135/201406.csv', '../data_in/41135/201412.csv', '../data_in/41135/200903.csv', '../data_in/41135/201002.csv', '../data_in/41135/201003.csv', '../data_in/41135/201201.csv', '../data_in/41135/200902.csv', '../data_in/41135/201407.csv', '../data_in/41135/201611.csv', '../data_in/41135/201605.csv', '../data_in/41135/201607.csv', '../data_in/41135/201808.csv', '../data_in/41135/201411.csv', '../data_in/41135/201405.csv', '../data_in/41135/201203.csv', '../data_in/41135/201001.csv', '../data_in/41135/201202.csv', '../data_in/41135/200901.csv', '../data_in/41135/201404.csv', '../data_in/41135/201410.csv', '../data_in/41135/201809.csv', '../data_in/41135/201606.csv', '../data_in/41135/201612.csv', '../data_in/41135/201602.csv', '../data_in/41135/200911.csv', '../data_in/41135/200905.csv', '../data_in/41135/201206.csv', '../

In [56]:
data = df[df['apt_name'] == apt_name_list[0]]
data = data[['apt_name','dedicated_area','floor','transaction_amount','transaction_date']]
data = data.sort_values('transaction_date', ascending=True).groupby(['floor','dedicated_area']).tail(1)

In [57]:
data

,apt_name,dedicated_area,floor,transaction_amount,transaction_date
3,샛별마을(라이프),153.24,13,125000.0,2006-01-12
0,샛별마을(라이프),89.80,16,58500.0,2006-02-01
7,샛별마을(라이프),78.52,24,48000.0,2006-03-04
6,샛별마을(라이프),153.24,17,120510.0,2007-12-21
12,샛별마을(라이프),84.99,19,58000.0,2009-05-13
...,...,...,...,...,...
34,샛별마을(라이프),84.99,5,115000.0,2020-11-26
37,샛별마을(라이프),85.00,9,123000.0,2020-11-27
5,샛별마을(라이프),84.99,24,108000.0,2020-12-08
6,샛별마을(라이프),85.00,12,127000.0,2020-12-09


In [51]:
data.sort_values('transaction_date', ascending=False).head(1)

,,,,transaction_date
apt_name,dedicated_area,floor,transaction_amount,
샛별마을(라이프),126.42,6,153000.0,2021-01-22


In [19]:
area_code_dirs = list(filter(lambda x: 'tar' not in x, glob.glob(os.path.join('../data_in','*'))))

In [63]:
trade_type = 'apt-trade'
for area_code_dir in area_code_dirs: #tqdm.tqdm(area_code_dirs): 
    area_code = area_code_dir.split('/')[-1]
    print(area_code_dir)
    filelist = glob.glob(os.path.join(area_code_dir, '*.csv'))
    frames = []
    print(filelist[:1])
    print(filelist)
    for filepath in tqdm.tqdm(filelist):
        frame = pd.read_csv(filepath, usecols=COLS[trade_type])
        frames.append(frame)
    print(len(df))
    df = pd.concat(frames, axis=0)
    df = preprocessing(df)
    print(df['apt_name'].unique().tolist())
    df = price_dedicatedarea_floor(df)
    df.to_csv(f'../data_out/price_dedicatedarea_floor/41135.csv', index=False)

10%|█         | 19/182 [00:00<00:00, 185.66it/s]../data_in/41135
['../data_in/41135/201604.csv']
['../data_in/41135/201604.csv', '../data_in/41135/201610.csv', '../data_in/41135/201406.csv', '../data_in/41135/201412.csv', '../data_in/41135/200903.csv', '../data_in/41135/201002.csv', '../data_in/41135/201003.csv', '../data_in/41135/201201.csv', '../data_in/41135/200902.csv', '../data_in/41135/201407.csv', '../data_in/41135/201611.csv', '../data_in/41135/201605.csv', '../data_in/41135/201607.csv', '../data_in/41135/201808.csv', '../data_in/41135/201411.csv', '../data_in/41135/201405.csv', '../data_in/41135/201203.csv', '../data_in/41135/201001.csv', '../data_in/41135/201202.csv', '../data_in/41135/200901.csv', '../data_in/41135/201404.csv', '../data_in/41135/201410.csv', '../data_in/41135/201809.csv', '../data_in/41135/201606.csv', '../data_in/41135/201612.csv', '../data_in/41135/201602.csv', '../data_in/41135/200911.csv', '../data_in/41135/200905.csv', '../data_in/41135/201206.csv', '..